In [1]:
!pip install ortools
#!pip install dtale

     |████████████████████████████████| 32.7MB 3.3MB/s 


In [2]:
import pandas as pd
import numpy as np
import collections
# Clone the entire repo.
!git clone -l -s https://github.com/mswastik/optimization.git cloned-repo
%cd cloned-repo

Cloning into 'cloned-repo'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 92 (delta 24), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (92/92), done.
/content/cloned-repo


In [3]:
#Importing Data
df=pd.read_excel('orders.xlsx')
ma=pd.read_excel('item master.xlsx')
bu=pd.read_excel('item master.xlsx',sheet_name='bulk')
pa=pd.read_excel('item master.xlsx',sheet_name='packing')

In [4]:
# Formating Data
df = df[df['Line']=='Tmmthpkhti Limi']
df = df[df['Plant']==1024]
del df['Line']
df['due'] = df['Dispatch Date'] - df['SO Date'].min()
df['due'] = df['due'].dt.days
df['sodate'] = df['SO Date'] - df['SO Date'].min()
df['sodate'] = df['sodate'].dt.days
df.reset_index(inplace=True,drop=True)
df['Key']=df['SO'].astype('str')+df['FG Code']
del df['Dispatch Date']
del df['SO Date']
del df['Description']
del df['Customer']
df['Order Qty'] = df[df['Order Qty']!=0]['Order Qty']*1000

In [5]:
# Creating Parameter Data
jobs =df['Key']
op =[1,2,3,4]  # Ignore as of now
mc = [1,2,3,4,5,6]
horizon = max(df["due"])+900
due = df.set_index('Key')
dur=df[['Key','FG Code','Order Qty']]
dur=dur.merge(pa[["FG Code","Line","Output"]], on="FG Code")
dur["Output"] = dur['Output']*100
dur["duration"]=dur["Order Qty"]/dur["Output"]
dur.set_index(['Key','Line'],inplace=True)
del dur['FG Code']
dur['Output'].replace(0,1,inplace=True)
df['Order Qty']=df['Order Qty'].fillna(0).astype('int')
dur['duration']=dur['duration'].fillna(0).astype('int')
dur['Output']=dur['Output'].fillna(0).astype('int')

In [ ]:
#Testing
#dur['duration'].loc[('17023287gd29904300iX',1)]

216

In [ ]:
#Exploring Data
 
from dtale import show
import dtale.app as dtale_app
dtale_app.USE_COLAB = True
show(df)

https://n6ekujl3anh-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/1

In [6]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()

In [13]:
task_type = collections.namedtuple('task_type', 'start end duration')
all_tasks = {}
for j in jobs:
    for m in mc:
        suffix = '_%s_%i' % (j,m)
        start = model.NewIntVar(0, horizon, "start"+suffix)
        end = model.NewIntVar(0, horizon, "end"+suffix)
        duration = model.NewIntervalVar(start, dur['duration'].loc[(j,m)], end, "duration"+suffix)
        all_tasks[(j, m)] = task_type(start=start, end=end, duration=duration)
 
mtj = {}
for m in mc:
    mjob=[]
    for j in jobs:
        mjob.append(all_tasks[(j, m)].duration)
    mtj[m] = mjob
    model.AddNoOverlap(mjob)
com = {}
for j in df['Key']:
  tt= 0
  for m in mc:
    #model.Add((all_tasks[(j, m)].end-all_tasks[(j, m)].start)*dur['Output'].loc[(j,m)] >= df[df["Key"]==j]['Order Qty'].values[0])
    comp = (all_tasks[(j,m)].end - all_tasks[(j,m)].start)*dur["Output"].loc[(j,m)]
    tt = tt + comp
  com[j] = tt
  model.Add(com[j] >= df[df["Key"]==j]['Order Qty'].values[0])
    # Remove 0 from packing output, it is causing error
 
delay = model.NewIntVar(0, horizon, 'delay')
#delay = 0
for j in df['Key']:
  dd=[]
  for m in mc:
    dd.append(due['due'].loc[j]-all_tasks[(j, m)].end)
  delay += max(dd)

In [14]:
model.Minimize(delay)
solver = cp_model.CpSolver()
solver.Solve(model)
solver.StatusName()

'INFEASIBLE'

In [ ]:
delay

SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(S

**Indices**  
j - job  
m - machine  

**Decision Variable**  
$s_{j,m}$ - start time of job j at machine m  
$e_{j,m}$ - end time of job j at machine m  
$d_{j,m}$ - duration of job j at machine m  

**Parameters**  
H - Horizon  
$d_j$ - delivery date of job j  
$q_j$ - order qty of job j  
$o_{j,m}$ - output per hour of job j at machine m  

**Objective**  
$ \sum_{j} d_j - max(e_{j,m})  \qquad \forall m \in mc$  

**Constraints**  
$ \sum_m e_{j,m} < s_{i,m} or s_{j,m} > e_{i,m} \qquad \forall j,i \in jobs$  
$ \sum_j o_{j,m} \times d_{j,m} >= q_j \qquad \forall m \in ma $
